In [1]:
import torch
import pandas as pd

torch.manual_seed(1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

train = pd.read_csv("../input/2021-ai-w4-p2/train.csv")
x_test = pd.read_csv("../input/2021-ai-w4-p2/test.csv")
submit = pd.read_csv("../input/2021-ai-w4-p2/sample.csv")

x_train = train.drop('label', axis = 1)
y_train = train['label']


x_train.shape

(1650, 7)

In [2]:
df = y_train
len(df.unique())

22

In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_train = le.fit_transform(y_train)


In [4]:
#표준화
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [5]:
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
x_test = torch.FloatTensor(x_test)

In [6]:
layer = torch.nn.Linear(7,22)
model = torch.nn.Sequential(layer).to(device)

loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
epochs = 10000

In [7]:
for e in range(epochs +1):
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    hypothesis = model(x_train)
    
    cost = loss(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if e % 1000 == 0:
        print("epoch : ", e , "cost : ", cost.item())

epoch :  0 cost :  3.3561909198760986
epoch :  1000 cost :  1.1059019565582275
epoch :  2000 cost :  0.6563892960548401
epoch :  3000 cost :  0.4564986526966095
epoch :  4000 cost :  0.33675575256347656
epoch :  5000 cost :  0.2564544677734375
epoch :  6000 cost :  0.199427992105484
epoch :  7000 cost :  0.1575988531112671
epoch :  8000 cost :  0.12632428109645844
epoch :  9000 cost :  0.10263720899820328
epoch :  10000 cost :  0.0845184251666069


In [8]:
x_test = x_test.to(device)
hypothesis = model(x_test)
predict = torch.argmax(hypothesis, dim = 1)

predict = le.inverse_transform(predict.cpu())

predict
    
   

array(['muskmelon', 'watermelon', 'papaya', 'papaya', 'apple', 'mango',
       'apple', 'mothbeans', 'mungbean', 'lentil', 'blackgram', 'coconut',
       'pomegranate', 'jute', 'coconut', 'pomegranate', 'apple', 'maize',
       'papaya', 'muskmelon', 'coffee', 'papaya', 'orange', 'papaya',
       'chickpea', 'jute', 'mungbean', 'orange', 'pigeonpeas', 'rice',
       'pomegranate', 'mothbeans', 'jute', 'lentil', 'jute', 'blackgram',
       'jute', 'chickpea', 'chickpea', 'kidneybeans', 'papaya', 'mango',
       'blackgram', 'maize', 'mungbean', 'maize', 'pigeonpeas', 'coconut',
       'muskmelon', 'maize', 'blackgram', 'coffee', 'grapes', 'mungbean',
       'coffee', 'kidneybeans', 'cotton', 'apple', 'banana', 'lentil',
       'watermelon', 'coconut', 'lentil', 'orange', 'papaya', 'blackgram',
       'orange', 'rice', 'muskmelon', 'pigeonpeas', 'muskmelon',
       'coconut', 'jute', 'banana', 'blackgram', 'papaya', 'banana',
       'cotton', 'watermelon', 'orange', 'coffee', 'chickpea',

In [9]:
submit['label'] = predict

In [10]:
submit

,id,label
0,0,muskmelon
1,1,watermelon
2,2,papaya
3,3,papaya
4,4,apple
...,...,...
545,545,coffee
546,546,papaya
547,547,chickpea
548,548,coffee


In [11]:
submit.to_csv("submission.csv", index = False)

s = pd.read_csv("submission.csv")
s

,id,label
0,0,muskmelon
1,1,watermelon
2,2,papaya
3,3,papaya
4,4,apple
...,...,...
545,545,coffee
546,546,papaya
547,547,chickpea
548,548,coffee
